In [1]:
#@title 0. 설정

HF_OLD_IMAGES = "pomupomu2/test_hbr_images"   # ✅ 기존 parquet 이미지 repo
HF_OLD_META   = "pomupomu2/test_hbr_meta"     # (지금은 안 씀: 이미지 v3 먼저)
HF_V3_IMAGES  = "pomupomu2/test_hbr_images_v3"  # ✅ 새 v3 zip repo
HF_PRIVATE    = True

# zip 샤딩 크기 (너무 크면 업로드/재시도 불편함)
# 200~800 권장. (이미지 평균 용량 크면 200~400)
ZIP_SHARD_SIZE = 400

# 코랩 임시 작업 폴더
WORK_DIR = "/content/hbr_v3_mig"

In [2]:
#@title 1. 설치 (버전 고정)

!pip -q install -U "huggingface-hub==0.34.4" "datasets>=2.20.0" "pyarrow>=14.0.0" "tqdm"
!pip -q install -U hf_transfer || true


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 16.7 MB/s eta 0:00:00


In [3]:
#@title 2. 토큰/환경 + v3 repo 생성

import os, json, math, shutil
from google.colab import userdata
from huggingface_hub import HfApi, create_repo

hf_token = userdata.get("HF_KEY")
if not hf_token:
    raise RuntimeError("HF_KEY 토큰을 찾지 못했습니다.")

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"  # 가능하면 빨라짐

api = HfApi(token=hf_token)
create_repo(HF_V3_IMAGES, repo_type="dataset", private=HF_PRIVATE, exist_ok=True, token=hf_token)

os.makedirs(WORK_DIR, exist_ok=True)
print("✅ ready")
print("src:", HF_OLD_IMAGES)
print("dst:", HF_V3_IMAGES)
print("WORK_DIR:", WORK_DIR)


✅ ready
src: pomupomu2/test_hbr_images
dst: pomupomu2/test_hbr_images_v3
WORK_DIR: /content/hbr_v3_mig


In [4]:
#@title 3. 기존 parquet images 로드 (decode=False)

from datasets import load_dataset
from datasets import Image as HFImage

ds_old = load_dataset(HF_OLD_IMAGES, split="train", token=hf_token)
# ✅ PIL 디코딩 금지: 원본 bytes를 그대로 쓰기 위해
ds_old = ds_old.cast_column("image", HFImage(decode=False))

print("rows:", len(ds_old))
print("columns:", ds_old.column_names)


README.md:   0%|          | 0.00/524 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

data/train-00000-of-00023.parquet:   0%|          | 0.00/779M [00:00<?, ?B/s]

data/train-00001-of-00023.parquet:   0%|          | 0.00/950M [00:00<?, ?B/s]

...

data/train-00021-of-00023.parquet:   0%|          | 0.00/477M [00:00<?, ?B/s]

data/train-00022-of-00023.parquet:   0%|          | 0.00/313M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]

rows: 5000
columns: ['md5', 'image', 'file_ext', 'file_size', 'first_seen_post_id', 'first_seen_at', 'source_file_url']


In [5]:
#@title 4. zip shard 생성 + 업로드 (원본 bytes 그대로)

import zipfile
from tqdm.auto import tqdm

DATA_DIR = os.path.join(WORK_DIR, "data")
os.makedirs(DATA_DIR, exist_ok=True)

def now_iso():
    import time
    return time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime())

def safe_md5(x):
    return (x or "").lower().strip()

def shard_name(k):
    return f"data/images-{k:05d}.zip"  # repo 안 경로

def make_local_zip_path(k):
    return os.path.join(DATA_DIR, f"images-{k:05d}.zip")

total = len(ds_old)
num_shards = math.ceil(total / ZIP_SHARD_SIZE)

print("total:", total)
print("ZIP_SHARD_SIZE:", ZIP_SHARD_SIZE)
print("num_shards:", num_shards)

uploaded = 0

for shard_idx in range(num_shards):
    start = shard_idx * ZIP_SHARD_SIZE
    end = min(total, (shard_idx + 1) * ZIP_SHARD_SIZE)

    local_zip = make_local_zip_path(shard_idx+1)
    repo_zip  = shard_name(shard_idx+1)

    # 이미 올렸으면 스킵하고 싶으면: (선택) repo에 존재여부 확인 로직 넣을 수 있음
    # 지금은 단순하게 매번 생성/업로드

    # zip 작성
    with zipfile.ZipFile(local_zip, "w", compression=zipfile.ZIP_STORED) as zf:
        for i in range(start, end):
            r = ds_old[i]
            md5 = safe_md5(r.get("md5"))
            if not md5:
                continue

            ext = (r.get("file_ext") or "jpg").lower().strip()
            if ext == "jpeg": ext = "jpg"

            img_obj = r["image"]  # decode=False => dict 형태 기대
            b = None
            if isinstance(img_obj, dict):
                b = img_obj.get("bytes", None)

            if not b:
                # bytes가 없으면 이 케이스는 "path 기반"일 수 있는데,
                # parquet embedded라면 보통 bytes가 있음.
                raise RuntimeError(f"Missing image bytes at row={i}, md5={md5}")

            # zip 내부 파일명: md5.ext (나중에 찾기 쉬움)
            zf.writestr(f"{md5}.{ext}", b)

    # HF 업로드(한 파일)
    api.upload_file(
        path_or_fileobj=local_zip,
        path_in_repo=repo_zip,
        repo_id=HF_V3_IMAGES,
        repo_type="dataset",
        token=hf_token,
    )

    uploaded += 1
    os.remove(local_zip)  # ✅ 디스크 안 쌓이게 즉시 삭제

    if (shard_idx+1) % 5 == 0 or shard_idx == num_shards-1:
        print(f"✅ uploaded shards: {uploaded}/{num_shards}  (rows {start}..{end-1})")

print("== DONE ==")
print("uploaded shards:", uploaded)


total: 5000
ZIP_SHARD_SIZE: 400
num_shards: 13


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...nt/hbr_v3_mig/data/images-00001.zip:   0%|          | 4.30MB / 1.49GB            

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

...

  ...nt/hbr_v3_mig/data/images-00012.zip:   0%|          | 2.51MB /  734MB            

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...nt/hbr_v3_mig/data/images-00013.zip:   3%|2         | 8.51MB /  299MB            

✅ uploaded shards: 13/13  (rows 4800..4999)
== DONE ==
uploaded shards: 13


In [6]:
#@title 5. v3 manifest 업로드 (md5, ext, shard)

from datasets import Dataset
from tqdm.auto import tqdm

rows = []
total = len(ds_old)

for i in tqdm(range(total), desc="manifest"):
    r = ds_old[i]
    md5 = safe_md5(r.get("md5"))
    if not md5:
        continue
    ext = (r.get("file_ext") or "jpg").lower().strip()
    if ext == "jpeg": ext = "jpg"
    shard_idx = (i // ZIP_SHARD_SIZE) + 1
    rows.append({
        "md5": md5,
        "file_ext": ext,
        "shard": shard_idx,
        "zip_path": shard_name(shard_idx),
        "inner_path": f"{md5}.{ext}",
        "file_size": int(r.get("file_size") or 0),
        "first_seen_post_id": int(r.get("first_seen_post_id") or 0),
        "first_seen_at": r.get("first_seen_at") or "",
        "source_file_url": r.get("source_file_url") or "",
    })

ds_manifest = Dataset.from_list(rows)
MANI_DIR = os.path.join(WORK_DIR, "manifest")
os.makedirs(MANI_DIR, exist_ok=True)

mani_parquet = os.path.join(MANI_DIR, "manifest.parquet")
mani_jsonl   = os.path.join(MANI_DIR, "manifest.jsonl")

ds_manifest.to_parquet(mani_parquet)
with open(mani_jsonl, "w", encoding="utf-8") as f:
    for r in rows:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

api.upload_file(
    path_or_fileobj=mani_parquet,
    path_in_repo="manifest/manifest.parquet",
    repo_id=HF_V3_IMAGES,
    repo_type="dataset",
    token=hf_token,
)
api.upload_file(
    path_or_fileobj=mani_jsonl,
    path_in_repo="manifest/manifest.jsonl",
    repo_id=HF_V3_IMAGES,
    repo_type="dataset",
    token=hf_token,
)

print("✅ manifest uploaded. rows:", len(rows))


manifest:   0%|          | 0/5000 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...br_v3_mig/manifest/manifest.parquet:  21%|##1       |  131kB /  616kB            

✅ manifest uploaded. rows: 5000


In [7]:
#@title meta v3 만들기 (기존 meta -> meta_v3 복사)

from google.colab import userdata
from huggingface_hub import create_repo
from datasets import load_dataset

hf_token = userdata.get("HF_KEY")
if not hf_token:
    raise RuntimeError("HF_KEY 토큰을 찾지 못했습니다.")

HF_OLD_META = "pomupomu2/test_hbr_meta"       # or test_hbr_meta_v2
HF_V3_META  = "pomupomu2/test_hbr_meta_v3"
HF_PRIVATE  = True

create_repo(HF_V3_META, repo_type="dataset", private=HF_PRIVATE, exist_ok=True, token=hf_token)

ds_meta = load_dataset(HF_OLD_META, split="train", token=hf_token)
ds_meta.push_to_hub(HF_V3_META, private=HF_PRIVATE, token=hf_token, split="train")

print("✅ meta_v3 done:", HF_V3_META, "rows:", len(ds_meta))


README.md:   0%|          | 0.00/957 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        : 100%|##########| 1.50MB / 1.50MB            

✅ meta_v3 done: pomupomu2/test_hbr_meta_v3 rows: 5000
